In [1]:
import tensorflow as tf
import numpy as np
import pathlib
tf.__version__

'2.0.0'

In [2]:
import tensorflow.keras.layers as layers

### Inception介绍
与NiN网络比，inception网络更加复杂

![Image text](http://zh.d2l.ai/_images/inception.svg)

由上图可以看出Inception有四条并行线路,前三条线路分别是1x1,3x3,5x5的卷积层来抽取，其中中间2个线路会对输入先做1×1卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用3×3最大池化层，后接1×1卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

In [52]:
class InceptionBlock(layers.Layer):
    def __init__(self,c1,c2,c3,c4,**kwargs):
        super(InceptionBlock,self).__init__(**kwargs)
        
        self.conv1 = layers.Conv2D(c1,kernel_size=1,activation="relu")
        
        self.conv2_1 = layers.Conv2D(c2[0],kernel_size=1,activation="relu")
        self.conv2_2 = layers.Conv2D(c2[1],kernel_size=3,activation="relu",padding="SAME")
        
        self.conv3_1 = layers.Conv2D(c3[0],kernel_size=1,activation="relu")
        self.conv3_2 = layers.Conv2D(c3[1],kernel_size=5,activation="relu",padding="SAME")
        
        self.conv4_1 = layers.ZeroPadding2D()
        self.conv4_2 = layers.MaxPooling2D(pool_size=(3,3),strides=1)
        self.conv4_3 = layers.Conv2D(c4,kernel_size=1,activation="relu")
        self.merge = layers.Concatenate(-1)
    
    def call(self,x):
        out1 = self.conv1(x)
        out2 = self.conv2_2(self.conv2_1(x))
        out3 = self.conv3_2(self.conv3_1(x))
        out4 = self.conv4_3(self.conv4_2(self.conv4_1(x)))
        return self.merge([out1,out2,out3,out4])

In [53]:
inception_block = InceptionBlock(3,[4,5],[6,7],8)

In [54]:
x = tf.random.normal(shape=(96,96,3))
x = tf.expand_dims(x,axis=0)
x.shape

TensorShape([1, 96, 96, 3])

In [55]:
y = inception_block(x)
y.shape

TensorShape([1, 96, 96, 23])

In [56]:
b1 = tf.keras.Sequential()
b1.add(layers.Conv2D(64,strides=(2,2),kernel_size=7,padding="SAME",activation="relu"))
b1.add(layers.MaxPooling2D(pool_size=3,strides=(2,2),padding="SAME"))

In [57]:
out1 = b1(x)
out1.shape

TensorShape([1, 24, 24, 64])

In [58]:
b2 = tf.keras.Sequential()
b2.add(layers.Conv2D(64,kernel_size=1,activation="relu"))
b2.add(layers.Conv2D(192,kernel_size=3,padding="SAME",activation="relu"))
b2.add(layers.MaxPooling2D(pool_size=3,strides=(2,2),padding="SAME"))

In [59]:
out2 = b2(out1)
out2.shape

TensorShape([1, 12, 12, 192])

In [60]:
b3 = tf.keras.Sequential()
b3.add(InceptionBlock(64,(96,128),(16,32),32))
b3.add(InceptionBlock(128,(128,192),(32,96),64))
b3.add(layers.MaxPooling2D(pool_size=3,strides=(2,2),padding="SAME"))

In [61]:
out3 = b3(out2)
out3.shape

TensorShape([1, 6, 6, 480])

In [62]:
b4 = tf.keras.Sequential()
b4.add(InceptionBlock(192,(96,208),(16,48),64))
b4.add(InceptionBlock(160,(112,224),(24,64),64))
b4.add(InceptionBlock(128,(128,256),(24,64),64))
b4.add(InceptionBlock(112,(144,288),(32,64),64))
b4.add(InceptionBlock(256,(160,320),(32,128),128))
b4.add(layers.MaxPooling2D(pool_size=3,strides=2,padding="same"))

In [63]:
out4 = b4(out3)
out4.shape

TensorShape([1, 3, 3, 832])

In [70]:
b5 =tf.keras.Sequential()
b5.add(InceptionBlock(256,(160,320),(32,128),128))
b5.add(InceptionBlock(384,(192,384),(48,128),128))
b5.add(layers.GlobalAvgPool2D())


In [71]:
out5 = b5(out4)
out5.shape

TensorShape([1, 1024])

In [74]:
net = tf.keras.Sequential([
   layers.InputLayer(input_shape=(224,224,3)), b1,b2,b3,b4,b5,layers.Flatten(),layers.Dense(5,activation="softmax")
])

In [75]:
net.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_11 (Sequential)   (None, 56, 56, 64)        9472      
_________________________________________________________________
sequential_12 (Sequential)   (None, 28, 28, 192)       114944    
_________________________________________________________________
sequential_13 (Sequential)   (None, 14, 14, 480)       552432    
_________________________________________________________________
sequential_14 (Sequential)   (None, 7, 7, 832)         2809168   
_________________________________________________________________
sequential_18 (Sequential)   (None, 1024)              2487536   
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)               

In [76]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 32

In [77]:
DATA_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(fname="flower_photos",origin=DATA_URL,untar=True)
import pathlib
data_dir = pathlib.Path(data_dir)
ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
class_lable =  np.array([ item.name for item in data_dir.glob("*") if item.name != "LICENSE.txt" ])
## 转换lable为数值型数据
lable_dic =dict( zip(class_lable,np.asarray(range(0,len(class_lable)))))
@tf.function
def get_lable(x):
    arr_str = tf.strings.split(x,sep="\\")
    return arr_str[-2] == class_lable

def decode_img(img):
    img = tf.image.decode_image(img,channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_with_crop_or_pad(img,IMG_WIDTH,IMG_HEIGHT)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img

def img_map_fun(x):
    lable = get_lable(x)
    img = tf.io.read_file(x)
    img = decode_img(img)
    return img,lable

lable = tf.constant([0,1,2,3,4])
@tf.function
def map_lable_fun(x,y):
    print(y)
    y = lable[y][0]
    return x,y

ds = ds.map(map_func=img_map_fun,num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds = ds.map(map_func=map_lable_fun)

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000,batch_size = 256):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
    else:
        ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(batch_size)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

ds = prepare_for_training(ds,batch_size = BATCH_SIZE)

image_batch, label_batch = next(iter(ds))

Tensor("y:0", shape=(5,), dtype=bool)


In [78]:
image_batch.shape

TensorShape([32, 224, 224, 3])

In [81]:
net.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=["acc"])
net.fit(ds,steps_per_epoch=100,epochs=5)

Train for 100 steps
Epoch 1/5
100/100 [==============================] - 934s 9s/step - loss: 2.1081 - acc: 0.2412
Epoch 2/5
100/100 [==============================] - 912s 9s/step - loss: 1.6573 - acc: 0.2716
Epoch 3/5
100/100 [==============================] - 912s 9s/step - loss: 1.4187 - acc: 0.3769
Epoch 4/5
100/100 [==============================] - 932s 9s/step - loss: 1.3856 - acc: 0.3816
Epoch 5/5
100/100 [==============================] - 896s 9s/step - loss: 1.3301 - acc: 0.4216
